In [1]:
import os

In [3]:
os.chdir("../")
%pwd

'c:\\ml step\\endtoendmlops\\Flask-E2E-Mlops'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [6]:
from src.FlaskMlops.constants import *
from src.FlaskMlops.utils.common import read_yaml,create_directories,save_json

In [13]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        
        
        create_directories([self.config.artifact_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN    
        
        create_directories([config.root_dir])
        
        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/PREMNATH1311/Flask-E2E-Mlops.mlflow"
        )
        return model_evaluation_config

In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [24]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config=config
        
    def eval_metrics(self,actual,pred):
        rmse=np.sqrt(mean_squared_error(actual,pred))
        mae=mean_absolute_error(actual,pred)
        r2=r2_score(actual,pred)
        
        return rmse,mae,r2
    
    def log_into_mlflow(self):
        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)
        
        test_x=test_data.drop([self.config.target_column],axis=1)
        test_y=test_data[[self.config.target_column]]
        
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        mlflow.set_registry_uri(self.config.mlflow_uri)

        mlflow.set_experiment("ElasticNet-Model-Evaluation")
        tracking_uri_type_store=urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities=model.predict(test_x)
            (rmse,mae,r2)=self.eval_metrics(test_y,predicted_qualities)
            
            print("Model evaluation metrics")
            print(f"RMSE:{rmse}")
            print(f"MAE:{mae}")
            print(f"R2:{r2}")
            scores={"rmse":rmse,"mae":mae,"r2":r2}
            save_json(path=Path(self.config.metric_file_name),data=scores)
            
            mlflow.log_param("model_type","ElasticNet")
            # mlflow.log_param("alpha",self.config.all_params.alpha)
            # mlflow.log_param("l1_ratio",self.config.all_params.l1_ratio)
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)
            if tracking_uri_type_store != "file":
    # Log model without registry to avoid 500 errors
                        mlflow.sklearn.log_model(
                        model,
                        name="ElasticNet"
                )
            else:
                mlflow.sklearn.log_model(
                    model,
                    name="ElasticNet"
                )
        
        
        

In [26]:
try:
    config=ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluation=ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-12-25 22:02:25,981: INFO: common: Yaml file: config\config.yaml loaded successfully]
[2025-12-25 22:02:26,002: INFO: common: Yaml file: params.yaml loaded successfully]
[2025-12-25 22:02:26,008: INFO: common: Yaml file: schema.yaml loaded successfully]
[2025-12-25 22:02:26,022: INFO: common: Created directory at: artifacts]
[2025-12-25 22:02:26,029: INFO: common: Created directory at: artifacts/model_evaluation]
Model evaluation metrics
RMSE:0.6948959327564913
MAE:0.5559299193365513
R2:0.2634527801075135
[2025-12-25 22:02:27,689: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json ]
🏃 View run worried-rat-705 at: https://dagshub.com/PREMNATH1311/Flask-E2E-Mlops.mlflow/#/experiments/0/runs/d539d36d67274763bcfd89f13bfdbb4b
🧪 View experiment at: https://dagshub.com/PREMNATH1311/Flask-E2E-Mlops.mlflow/#/experiments/0
